In [ ]:
from os.path import join
import matplotlib.pyplot as plt
import earthaccess

import geopandas as gpd
import rasters as rt

from EMITL2ARFL import *

date range

In [ ]:
start_date = "2023-04-15"
end_date = "2023-04-15"

download directory

In [ ]:
download_directory = "~/data/EMIT_download"

output directory

In [ ]:
output_directory = "~/data/Kings Canyon EMIT"

Upper Kings area of interest

In [ ]:
gdf = gpd.read_file("upper_kings.kml")
gdf

In [ ]:
gdf.geometry[0]

In [ ]:
bbox_UTM = rt.Polygon(gdf.unary_union).UTM.bbox
bbox_UTM

In [ ]:
grid = rt.RasterGrid.from_bbox(bbox_UTM, cell_size=60, crs=bbox_UTM.crs)
grid

Log into `earthaccess` using `netrc` credentials

In [ ]:
earthaccess.login(strategy="netrc", persist=True)

search for EMIT granules covering area of interest

In [ ]:
search_results = search_EMIT_L2A_RFL_granules(
    start_UTC=start_date,
    end_UTC=end_date,
    geometry=grid
)

search_results

download each granule, extract subset projected to target grid, and write to multi-band GeoTIFF

In [ ]:
# for search_result in search_results:
#     granule_ID = dict(search_result.items())["meta"]["native-id"]
#     print(granule_ID)
#     output_filename = join(output_directory, f"{granule_ID}.tif")
#     granule = retrieve_EMIT_L2A_RFL_granule(search_result)
#     reflectance = granule.reflectance(geometry=grid)

#     display(reflectance.imshow())

#     print(output_filename)
#     reflectance.to_geotiff(output_filename)

In [ ]:
search_result = search_results[0]

granule_ID = dict(search_result.items())["meta"]["native-id"]
print(granule_ID)
output_filename = join(output_directory, f"{granule_ID}.tif")
granule = retrieve_EMIT_L2A_RFL_granule(search_result)
reflectance = granule.reflectance(geometry=grid)

reflectance

In [ ]:
qmask = granule.quality_mask(geometry=grid)
qmask

In [ ]:
reflectance.shape

In [ ]:
qmask.shape

In [ ]:
qmask.dtype

In [ ]:
masked_reflectance = reflectance.mask(~qmask)
masked_reflectance

In [ ]:
type(qmask.geometry)

In [ ]:
scene_reflectance = granule.reflectance()
scene_reflectance

In [ ]:
search_result_other = search_results[1]

granule_ID_other = dict(search_result.items())["meta"]["native-id"]
print(granule_ID_other)
output_filename_other = join(output_directory, f"{granule_ID_other}.tif")
granule_other = retrieve_EMIT_L2A_RFL_granule(search_result_other)
reflectance_other = granule_other.reflectance(geometry=grid)

reflectance_other

In [ ]:
mosaic = rt.mosaic([reflectance, reflectance_other], geometry=grid)
mosaic